### kNN
### Ordinal Regression - binary crossentropy + metrics=['accuracy']
### Oversampling 3x 4 + 4x 5

In [1]:
from keras.engine import Model
from keras.layers import Flatten, Dense, Input, Dropout
from keras_vggface.vggface import VGGFace
from keras.preprocessing.image import ImageDataGenerator
from keras_vggface import utils
from keras.optimizers import Adam
from keras.preprocessing import image
from PIL import Image
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.utils import class_weight
import keras
import random, string
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from skimage.io import imread_collection
import scipy.misc
%matplotlib inline

/home/gloria/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/gloria/tensorflow/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# Constants
IMG_SIZE = 224

#IMG_DIR = '../project/all_females_AUG_2_combined_oversampled_train_s'
#RATING_PATH = '../project/all_females_AUG_2_combined_oversampled_train_s/ratings.txt'

IMG_DIR = '../project/all_females_combined_train_s'
RATING_PATH = '../project/all_females_combined_train_s/train_ratings.txt'

VAL_IMG_DIR = '../project/all_females_combined_val_s'
VAL_RATING_PATH = '../project/all_females_combined_val_s/val_ratings.txt'


VERSION = 'v6-val-oversampled'

In [ ]:
def getMean(train_data):
    #calculate mean values for pixels (RGB)

    data = np.zeros((train_data.shape[0], IMG_SIZE, IMG_SIZE, 3)) 
    for i in range(train_data.shape[0]):#train_data:
        _img = image.load_img(os.path.join(IMG_DIR, train_data[i]), target_size=(IMG_SIZE,IMG_SIZE))
        data[i,:,:,:] = image.img_to_array(_img)

    mean = np.mean(data, axis=(0, 1, 2))
    print("Pixel means: ", mean)
    return mean

In [ ]:
def preprocess(x,mean):
    x[:,:,0] -= mean[0]
    x[:,:,1] -= mean[1]
    x[:,:,2] -= mean[2]

In [3]:
def prepareData(percentage):
    # Load ratings
    ratings = np.genfromtxt(RATING_PATH)
    
    # Convert ratings to the desired format
    # 1. First round them
    ratings_rounded = np.round(ratings, 1).astype(np.int8)

    # 2. Then encode as suggested by Cheng (2007)
    ratings_prepared = np.zeros((len(ratings_rounded), len(np.unique(ratings_rounded)))).astype(np.int8)
    for i, r in enumerate(ratings_rounded):
        for j in range(r):
            ratings_prepared[i, j] = 1
        
    # 3. Finally, make sure assignment is correct
    assert np.all(np.sum(ratings_prepared, axis=1).astype(np.int8) == ratings_rounded)
    
    ### Move all images according to ratings
    images = sorted(os.listdir(IMG_DIR))
    #remove train_ratings.txt from images
    images.remove('train_ratings.txt')
    #convert to np.array for indexing
    images = np.array(images)

    ### Make sure lengths of ratings and images correspond 
    print(len(ratings))
    print(len(images))
    assert len(ratings) == len(images)
    
    perm_ratings = np.random.permutation(len(ratings))
    test_len = int(len(ratings) * percentage/100)
    train_len = len(ratings) - test_len

    test_ind = perm_ratings[:test_len]
    train_ind = perm_ratings[test_len:]
    
    #test_ind = test_ind.astype(np.int8)
    #print(train_ind)# = train_ind.astype(np.int8)
    
    #get test & train data
    data_test = np.zeros((len(test_ind), IMG_SIZE, IMG_SIZE, 3))    
    data_train = np.zeros((len(train_ind), IMG_SIZE, IMG_SIZE, 3)) 
    last_set_train = 0
    last_set_test = 0
    
    mean = getMean(images[train_ind])
    #print(mean)
    
    for idx, _im in enumerate(images): #iterate over images
        if idx in test_ind:
            _img = image.load_img(os.path.join(IMG_DIR, _im), target_size=(IMG_SIZE,IMG_SIZE))
            _x = image.img_to_array(_img)
            _x = np.expand_dims(_x, axis=0)
            data_test[last_set_test, :, :, :] = preprocess(_x,mean)#utils.preprocess_input(_x, version=1) / 255.0
            last_set_test += 1
        if idx in train_ind:
            _img = image.load_img(os.path.join(IMG_DIR, _im), target_size=(IMG_SIZE,IMG_SIZE))
            _x = image.img_to_array(_img)
            _x = np.expand_dims(_x, axis=0)
            data_train[last_set_train, :, :, :] = preprocess(_x,mean)#utils.preprocess_input(_x, version=1) / 255.0
            last_set_train += 1
    

    return data_train, ratings_prepared[train_ind], data_test, ratings_prepared[test_ind], ratings_rounded, images[train_ind]

In [4]:
def prepareValData():
    # Load ratings
    ratings = np.genfromtxt(VAL_RATING_PATH)
    
    # Convert ratings to the desired format
    # 1. First round them
    ratings_rounded = np.round(ratings, 1).astype(np.int8)

    # 2. Then encode as suggested by Cheng (2007)
    ratings_prepared = np.zeros((len(ratings_rounded), len(np.unique(ratings_rounded))))
    for i, r in enumerate(ratings_rounded):
        for j in range(r):
            ratings_prepared[i, j] = 1
        
    # 3. Finally, make sure assignment is correct
    assert np.all(np.sum(ratings_prepared, axis=1).astype(np.int8) == ratings_rounded)
    
    ### Move all images according to ratings
    images = sorted(os.listdir(VAL_IMG_DIR))
    #remove val_ratings.txt from images
    images.remove('val_ratings.txt')
    #convert for indexing
    images = np.array(images)

    ### Make sure lengths of ratings and images correspond 
    assert len(ratings) == len(images)
    
    data = np.zeros((len(images), IMG_SIZE, IMG_SIZE, 3)) 
    last_set = 0
    
    for idx, _im in enumerate(images): #iterate over images
        _img = image.load_img(os.path.join(VAL_IMG_DIR, _im), target_size=(IMG_SIZE,IMG_SIZE))
        _x = image.img_to_array(_img)
        _x = np.expand_dims(_x, axis=0)
        data[last_set, :, :, :] = utils.preprocess_input(_x, version=1) / 255.0
        last_set += 1
    
    return data, ratings, images

In [5]:
def recommend(images, pred_classes):
    recs = np.where(pred_classes >= 4)
    
    for i in recs[0]:
        img = mpimg.imread(os.path.join(IMG_DIR, images[i]))
        imgplot = plt.imshow(img)
        plt.show()
        plt.clf()

def recommendVal(images, pred_classes):
    recs = np.where(pred_classes >= 4)
    
    for i in recs[0]:
        img = mpimg.imread(os.path.join(VAL_IMG_DIR, images[i]))
        imgplot = plt.imshow(img)
        plt.show()
        plt.clf()

In [6]:
#get augmented data from directory
x_train, y_train, x_test, y_test, ratings, images = prepareData(5)

681
681


In [7]:
print(x_train.shape)
print(x_test.shape)

(647, 224, 224, 3)
(34, 224, 224, 3)


In [8]:
# get features from vgg-face as input for kNN
model = VGGFace(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3), pooling='max')


# Freeze vgg layers
for layer in model.layers:
    layer.trainable = False
    
# Compute class weights since we have unbalaned classes
class_weight = class_weight.compute_class_weight('balanced', 
                                                 np.unique(ratings), 
                                                 ratings)



In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

In [10]:
features = model.predict(x_train)
print(features.shape)

(647, 512)


In [19]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(features,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [20]:
predictions = knn.predict(model.predict(x_test))
predictions = predictions.sum(axis=-1)
print(predictions)
print(y_test.sum(axis=-1))

confusion_matrix(y_test.sum(axis=-1),predictions)

[3 1 1 1 1 2 1 1 2 2 2 2 3 1 1 1 2 3 1 2 2 2 1 3 2 1 2 2 1 2 2 1 3 2]
[3 3 2 4 4 3 1 1 2 0 4 4 2 2 4 3 2 1 0 1 3 0 5 0 1 2 1 0 3 3 1 0 3 3]


array([[0, 2, 3, 1, 0, 0],
       [0, 2, 4, 1, 0, 0],
       [0, 3, 2, 1, 0, 0],
       [0, 3, 4, 2, 0, 0],
       [0, 3, 2, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]])

In [21]:
#Validation set:
x_val, ratings_val, images = prepareValData()

In [22]:
predictions = knn.predict(model.predict(x_val))
predictions = predictions.sum(axis=-1)
print(predictions)
ratings_val = np.array(ratings_val).astype(np.int8)
print(ratings_val)

confusion_matrix(ratings_val,predictions)

[3 1 2 1 1 1 2 2 1 2 1 1 2 2 1 2 2 1 2 2 2 2 1 2 2 2 1 3 1 2 1 1 1 2 3 2 2]
[0 0 0 0 0 0 0 1 1 1 1 1 1 1 2 2 2 2 2 2 2 3 3 3 3 3 3 3 4 4 4 4 4 4 4 5 5]


array([[0, 4, 2, 1, 0, 0],
       [0, 3, 4, 0, 0, 0],
       [0, 2, 5, 0, 0, 0],
       [0, 2, 4, 1, 0, 0],
       [0, 4, 2, 1, 0, 0],
       [0, 0, 2, 0, 0, 0]])

### Now train knn directly on images instead of features.

In [23]:
knn_img = KNeighborsClassifier(n_neighbors=10)
knn_img.fit(x_train.reshape((x_train.shape[0],-1)),y_train)

predictions = knn_img.predict(x_test.reshape((x_test.shape[0],-1)))
predictions = predictions.sum(axis=-1)

print(predictions)
print(y_test.sum(axis=-1))

confusion_matrix(y_test.sum(axis=-1),predictions)

[1 1 2 2 2 1 2 1 2 1 1 3 2 1 2 2 2 2 2 1 2 1 2 3 2 2 2 1 1 2 1 2 2 1]
[3 3 2 4 4 3 1 1 2 0 4 4 2 2 4 3 2 1 0 1 3 0 5 0 1 2 1 0 3 3 1 0 3 3]


array([[0, 3, 2, 1, 0, 0],
       [0, 3, 4, 0, 0, 0],
       [0, 1, 5, 0, 0, 0],
       [0, 5, 4, 0, 0, 0],
       [0, 1, 3, 1, 0, 0],
       [0, 0, 1, 0, 0, 0]])

In [24]:
#Validation set
predictions = knn_img.predict(x_val.reshape((x_val.shape[0],-1)))
predictions = predictions.sum(axis=-1)

print(predictions)
ratings_val = np.array(ratings_val).astype(np.int8)
print(ratings_val)


confusion_matrix(ratings_val,predictions)

[1 1 2 2 2 2 2 1 2 1 1 1 1 3 1 2 1 2 2 1 1 1 1 2 2 1 3 1 1 3 2 1 2 1 2 1 1]
[0 0 0 0 0 0 0 1 1 1 1 1 1 1 2 2 2 2 2 2 2 3 3 3 3 3 3 3 4 4 4 4 4 4 4 5 5]


array([[0, 2, 5, 0, 0, 0],
       [0, 5, 1, 1, 0, 0],
       [0, 4, 3, 0, 0, 0],
       [0, 4, 2, 1, 0, 0],
       [0, 3, 3, 1, 0, 0],
       [0, 2, 0, 0, 0, 0]])

### Better results with n_neighbors==5